# Global Uncertainty Analysis: Polynomial Chaos Expansion (PCE) for Chemical Reaction Systems


This ipython notebook uses MUQ as a basis for adaptive Polynomial Chaos Expansions to perform global uncertainty analysis for chemical reaction systems.  This ipython notebook details a workflow using RMG, Cantera, and MUQ codes.

Muq binary only works on linux systems, please also add the ~/anaconda/envs/your_env/lib folder to your $PYTHONPATH to import muq smoothly.

In [ ]:
import random

from rmgpy.tools.canteraModel import Cantera, getRMGSpeciesFromUserSpecies
from rmgpy.species import Species
from rmgpy.chemkin import loadChemkinFile
from rmgpy.tools.muq import ReactorPCEFactory
from rmgpy.tools.uncertainty import Uncertainty

## Initial setup

This section sets up everything needed to perform the global uncertainty analysis. This includes creating an instance of the Uncertainty class, loading the model to be analyzed, and setting up the Cantera reactor simulator.

In [ ]:
# Must use annotated chemkin file
chemkinFile = './data/pdd_model/chem_annotated.inp'
dictFile = './data/pdd_model/species_dictionary.txt'

In [ ]:
# Set output directory (Note: Global uncertainty analysis doesn't actually write any output files currently)
outputDirectory = './temp/uncertainty'

In [ ]:
# Initialize the Uncertainty class instance and load the model
uncertainty = Uncertainty(outputDirectory=outputDirectory)
uncertainty.loadModel(chemkinFile, dictFile)

In [ ]:
# Map the species to the objects within the Uncertainty class
PDD = Species().fromSMILES("CCCCCCCCCCCCc1ccccc1")
C11ene=Species().fromSMILES("CCCCCCCCCC=C")
ETHBENZ=Species().fromSMILES("CCc1ccccc1")
mapping = getRMGSpeciesFromUserSpecies([PDD,C11ene,ETHBENZ], uncertainty.speciesList)

# Define the reaction conditions
reactorTypeList = ['IdealGasConstPressureTemperatureReactor']
molFracList = [{mapping[PDD]: 1.0}]
Tlist = ([623],'K')
Plist = ([350],'bar')
reactionTimeList = ([72], 'h')

Global uncertainty analysis works by simulating the full model at random points within the uncertainty distributions of the input parameters. In the current implementation, the simulation is performed by Cantera, which we set up here using the RMG wrapper class.

In [ ]:
# Create the cantera model
job = Cantera(speciesList=uncertainty.speciesList, reactionList=uncertainty.reactionList, outputDirectory=outputDirectory)
# Load the cantera model based on the RMG reactions and species
job.loadModel()
# Generate the conditions based on the settings we declared earlier
job.generateConditions(reactorTypeList, reactionTimeList, molFracList, Tlist, Plist)

Next, we need to load the RMG-database into the Uncertainty instance which was created in order to extract the original sources for every estimated parameter in the model.

In [ ]:
uncertainty.loadDatabase(
    thermoLibraries=['DFT_QCI_thermo', 'primaryThermoLibrary'],
    kineticsFamilies='default',
    reactionLibraries=[],
)
uncertainty.extractSourcesFromModel()

## Part 1: Global uncertainty analysis for uncorrelated parameters

In [ ]:
# Assign uncorrelated parameter uncertainties 
uncertainty.assignParameterUncertainties(correlated=False)

Input a set of kinetic $(k)$ and thermo $(G)$ parameters to be propagated and their uncertainties $(\Delta\ln k, \Delta G)$ into the `ReactorPCEFactory` class. These kinetic and thermo parameters should typically be pre-screened from local uncertainty analysis to narrow down to the most influential parameters.

Parameter uncertainties are assigned the same way as for local uncertainty analysis and are provided directly from the `Uncertainty` instance.

Random sampling from the uncertainty distributions of the input parameters is aided by a set uncertainty factors, $f$, calculated from the input uncertainties $(\Delta\ln k, \Delta G)$, and a set of unit random variables, $\xi$, sampled from a uniform distribution.

For thermochemistry,

$$f^G = G_{max} - G_0 = G_{0} - G_{min} = \sqrt{3} \Delta G$$

$$G = \xi f^G_{n} + G_{0}$$

For kinetics,

$$f^k = \log_{10} \left(\frac{k_{max}}{k_0}\right) = \log_{10} \left(\frac{k_0}{k_{min}}\right) = \frac{\sqrt{3}}{\ln 10} \Delta \ln k$$

$$k = 10^{\xi f_{m}} k_{0}$$

This allows calculation of a new parameter value given the nominal value, standard deviation, and the random variable.

The MIT Uncertainty Quantification Library (MUQ) is used to perform the random sampling and construct a Polynomial Chaos Expansion (PCE) to fit the output variable of interest, mole fractions.

In [ ]:
# Choose input parameters to vary within their uncertainty bounds
kParams = [28, 26]  # RMG indices of reactions to vary
gParams = [1, 46]  # RMG indices of species to vary

In [ ]:
# Create ReactorPCEFactory global uncertainty analysis object for the uncorrelated case
reactorPCEFactory = ReactorPCEFactory(
    cantera=job,
    outputSpeciesList=[mapping[PDD], mapping[C11ene]],
    kParams=kParams,
    kUncertainty=uncertainty.kineticInputUncertainties,   
    gParams=gParams,
    gUncertainty=uncertainty.thermoInputUncertainties,
    correlated=False,
)

Begin generating the PCEs adaptively based a runtime.

There are actually three methods for generating PCEs. See the `ReactorPCEFactory.generatePCE` function for more details.

- Option 1: Adaptive for a pre-specified amount of time
- Option 2: Adaptively construct PCE to error tolerance
- Option 3: Used a fixed order, and (optionally) adapt later.  

In [ ]:
reactorPCEFactory.generatePCE(runTime=60)  # runtime of 60 seconds.

Let's compare the outputs for a test point using the real model versus using the PCE approximation.
Evaluate the desired output mole fractions based on a set of inputs `ins = [[ln(k)_rv], [G_rv]]` which contains the 
random unit uniform variables attributed to the uncertain kinetics and free energy parameters, respectively.

In [ ]:
# Create a random test point of length = number of kParams + number of gParams
randomTestPoint = [random.uniform(-1.0,1.0) for i in range(len(kParams)+len(gParams))]
trueTestPointOutput, pceTestPointOutput = reactorPCEFactory.compareOutput(randomTestPoint, log=False)

Obtain the results: the species mole fraction mean and variance computed from the PCE, as well as the global sensitivity indices.

In [ ]:
mean, variance, covariance, mainSens, totalSens = reactorPCEFactory.analyzeResults(log=False)

## Part 2: Global uncertainty analysis of correlated parameters

In [ ]:
uncertainty.assignParameterUncertainties(correlated=True)

In [ ]:
kParams = [
    'R_Addition_MultipleBond Cds-HH_Cds-Cs\H3/H;CsJ-CsHH',
    'Estimation BENZYL(58)+C11ene(46)=RAD3(16)',
]
gParams = [
    'Estimation PDD(1)',
    'Estimation C11ene(46)',
]

In [ ]:
reactorPCEFactoryCorrelated = ReactorPCEFactory(
    cantera=job,
    outputSpeciesList=[mapping[PDD], mapping[C11ene]],
    kParams=kParams,
    kUncertainty=uncertainty.kineticInputUncertainties,   
    gParams=gParams,
    gUncertainty=uncertainty.thermoInputUncertainties,
    correlated=True   
)

Do the same analysis for the correlated `reactorPCEFactory`

In [ ]:
reactorPCEFactoryCorrelated.generatePCE(runTime=60)  # runtime of 60 seconds.

In [ ]:
randomTestPoint = [random.uniform(-1.0,1.0) for i in range(len(kParams)+len(gParams))]
trueTestPointOutput, pceTestPointOutput = reactorPCEFactoryCorrelated.compareOutput(randomTestPoint, log=False)

In [ ]:
mean, variance, covariance, mainSens, totalSens = reactorPCEFactoryCorrelated.analyzeResults(log=False)